In [1]:
'''
Imports
'''
import IPython.display as ipd
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display
import ast
import os
import csv
from tinytag import TinyTag

C:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Data loading and preprocessing

## As first thing, we load the csv files

In [13]:
def get_mfcc(y, sr):
    mfcc = np.array(librosa.feature.mfcc(y=y, sr=sr))
    return mfcc

def get_melspectrogram(y, sr):
    melspectrogram = np.array(librosa.feature.melspectrogram(y=y, sr=sr))
    return melspectrogram

def get_chroma_stft(y, sr):
    chroma_stft = np.array(librosa.feature.chroma_stft(y=y, sr=sr))
    return chroma_stft

def get_tonnetz(y, sr):
    tonnetz = np.array(librosa.feature.tonnetz(y=y, sr=sr))
    return tonnetz

def get_chroma_cqt(y, sr):
    chroma_cqt = np.array(librosa.feature.chroma_cqt(y=y, sr=sr))
    return chroma_cqt

def get_chroma_cens(y, sr):
    chroma_cens = np.array(librosa.feature.chroma_cens(y=y, sr=sr))
    return chroma_cens

def get_spectral_centroid(y, sr):
    spectral_centroid = np.array(librosa.feature.spectral_centroid(y=y, sr=sr))
    return spectral_centroid

def get_spectral_bandwidth(y, sr):
    spectral_bandwidth = np.array(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    return spectral_bandwidth

def get_spectral_contrast(y, sr):
    spectral_contrast = np.array(librosa.feature.spectral_contrast(y=y, sr=sr))
    return spectral_contrast

def get_spectral_flatness(y):
    spectral_flatness = np.array(librosa.feature.spectral_flatness(y=y))
    return spectral_flatness

def get_spectral_rolloff(y, sr):
    spectral_rolloff = np.array(librosa.feature.spectral_rolloff(y=y, sr=sr))
    return spectral_rolloff

def get_poly_features(y, sr):
    poly_features = np.array(librosa.feature.poly_features(y=y, sr=sr))
    return poly_features

def get_rms(y):
    rms = np.array(librosa.feature.rms(y=y))
    return rms

def get_zero_crossing_rate(y):
    zero_crossing_rate = np.array(librosa.feature.zero_crossing_rate(y=y))
    return zero_crossing_rate

def get_tempogram(y, sr):
    tempogram = np.array(librosa.feature.tempogram(y=y, sr=sr))
    return tempogram

def get_fourier_tempogram(y, sr):
    fourier_tempogram = np.array(librosa.feature.fourier_tempogram(y=y, sr=sr))
    return fourier_tempogram

def calculate_metrics(array):
    features = []
    mean = np.mean(array)
    value_min = np.min(array)
    value_max = np.max(array)
    median = np.median(array)
    std = np.std(array)
    skew = scipy.stats.skew(array, axis=None)
    kurtosis = scipy.stats.kurtosis(array, axis=None)
    features.append(mean)
    features.append(value_min)
    features.append(value_max)
    features.append(median)
    features.append(std)
    features.append(skew)
    features.append(kurtosis)
    return features


def get_features(writer):
    
    i = 0
    for dirpath, dirnames, filenames in os.walk("dataset"):
        for filename in filenames:
            if ".mp3" in filename:
                file_path = dirpath+"\\"+filename
                #print(file_path)
                try:
                    tag = TinyTag.get(file_path)
                    row = [filename, tag.title, tag.artist, tag.album]
                    try:
                        y, sr = librosa.load(file_path, offset=0, duration=30)
                        row.extend(calculate_metrics(get_chroma_stft(y, sr)))
                        row.extend(calculate_metrics(get_chroma_cens(y, sr)))
                        row.extend(calculate_metrics(get_chroma_cqt(y, sr)))
                        row.extend(calculate_metrics(get_fourier_tempogram(y, sr)))
                        row.extend(calculate_metrics(get_melspectrogram(y, sr)))
                        row.extend(calculate_metrics(get_mfcc(y, sr)))
                        row.extend(calculate_metrics(get_poly_features(y, sr)))
                        row.extend(calculate_metrics(get_rms(y)))
                        row.extend(calculate_metrics(get_spectral_bandwidth(y, sr)))
                        row.extend(calculate_metrics(get_spectral_centroid(y, sr)))
                        row.extend(calculate_metrics(get_spectral_contrast(y, sr)))
                        row.extend(calculate_metrics(get_spectral_flatness(y)))
                        row.extend(calculate_metrics(get_spectral_rolloff(y, sr)))
                        row.extend(calculate_metrics(get_tempogram(y, sr)))
                        row.extend(calculate_metrics(get_tonnetz(y, sr)))
                        row.extend(calculate_metrics(get_zero_crossing_rate(y)))
                        row.append(tag.genre)
                        writer.writerow(row)
                    except:
                        print("librosa error: " + file_path)
                except:
                    print("tinyTag error: " + file_path)
                
                if (i%100 == 0):
                    print(i)
                i = i +1
        
def writeHeader(writer):
    header = ["filename", "title", "artist", "album"]
    features = ["chroma_stft", "chroma_cens", "chroma_cqt", "fourier_tempogram", "melspectrogram", "mfcc", "poly_features",
                "rms", "spectral_bandwidth", "spectral_centroid", "spectral_contrast", "spectral_flatness",
                "spectral_rolloff", "tempogram", "tonnetz", "zero_crossing_rate"]
    moments = ["mean", "min", "max", "median", "std", "skew", 'kurtosis']
    for feature in features:
        for moment in moments:
            header.append(feature + "_" + moment)
    header.append("genre")
    writer.writerow(header)
    
def main():
    # open the file in the write mode
    f = open('csv_file.csv', 'w', newline='')
    #f = open('csv_file.csv', 'a', newline='')

    # create the csv writer
    writer = csv.writer(f)

    # write header to the csv file
    writeHeader(writer)

    # write data to the csv file
    get_features(writer)

    # close the file
    f.close()
    
main()

0
tinyTag error: dataset\fma_medium\016\016428.mp3
tinyTag error: dataset\fma_medium\016\016431.mp3
librosa error: dataset\fma_medium\016\016521.mp3
librosa error: dataset\fma_medium\016\016522.mp3
librosa error: dataset\fma_medium\016\016523.mp3
librosa error: dataset\fma_medium\016\016524.mp3
librosa error: dataset\fma_medium\016\016525.mp3
librosa error: dataset\fma_medium\016\016526.mp3
librosa error: dataset\fma_medium\016\016527.mp3
librosa error: dataset\fma_medium\016\016528.mp3
librosa error: dataset\fma_medium\016\016529.mp3
librosa error: dataset\fma_medium\016\016530.mp3
librosa error: dataset\fma_medium\016\016531.mp3
librosa error: dataset\fma_medium\016\016532.mp3
librosa error: dataset\fma_medium\016\016533.mp3
100
200
300
400
500
600
700
800
900
librosa error: dataset\fma_medium\021\021891.mp3
librosa error: dataset\fma_medium\021\021892.mp3
librosa error: dataset\fma_medium\021\021893.mp3
librosa error: dataset\fma_medium\021\021894.mp3
librosa error: dataset\fma_medi

C:\ProgramData\Anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=554
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=384 is too small for input signal of length=70
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=384 is too small for input signal of length=269
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=704
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=352
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=1024 is too small for input signal of length=176
  return f(

librosa error: dataset\fma_medium\098\098571.mp3
12900
13000
librosa error: dataset\fma_medium\099\099134.mp3
13100
librosa error: dataset\fma_medium\099\099648.mp3
13200
librosa error: dataset\fma_medium\100\100535.mp3
librosa error: dataset\fma_medium\101\101316.mp3
librosa error: dataset\fma_medium\101\101928.mp3
librosa error: dataset\fma_medium\101\101929.mp3
librosa error: dataset\fma_medium\101\101930.mp3
13300
librosa error: dataset\fma_medium\102\102178.mp3
librosa error: dataset\fma_medium\102\102179.mp3
librosa error: dataset\fma_medium\102\102181.mp3
librosa error: dataset\fma_medium\102\102182.mp3
librosa error: dataset\fma_medium\102\102184.mp3
librosa error: dataset\fma_medium\102\102185.mp3
librosa error: dataset\fma_medium\102\102186.mp3
librosa error: dataset\fma_medium\102\102187.mp3
librosa error: dataset\fma_medium\102\102188.mp3
librosa error: dataset\fma_medium\102\102189.mp3
librosa error: dataset\fma_medium\102\102190.mp3
13400
librosa error: dataset\fma_medium

20500
librosa error: dataset\fma_medium\141\141970.mp3
librosa error: dataset\fma_medium\141\141990.mp3
20600
20700
librosa error: dataset\fma_medium\143\143139.mp3
librosa error: dataset\fma_medium\143\143144.mp3
librosa error: dataset\fma_medium\143\143145.mp3
librosa error: dataset\fma_medium\143\143146.mp3
20800
librosa error: dataset\fma_medium\143\143992.mp3
20900
librosa error: dataset\fma_medium\144\144589.mp3
librosa error: dataset\fma_medium\144\144809.mp3
21000
librosa error: dataset\fma_medium\145\145129.mp3
librosa error: dataset\fma_medium\145\145563.mp3
librosa error: dataset\fma_medium\145\145565.mp3
librosa error: dataset\fma_medium\145\145567.mp3
librosa error: dataset\fma_medium\145\145570.mp3
librosa error: dataset\fma_medium\145\145571.mp3
21100
librosa error: dataset\fma_medium\145\145817.mp3
21200
21300
21400
librosa error: dataset\fma_medium\147\147804.mp3
librosa error: dataset\fma_medium\147\147805.mp3
librosa error: dataset\fma_medium\147\147806.mp3
librosa e

In [17]:
y, sr = librosa.load("dataset\fma_medium\102\102185.mp3", offset=0, duration=30)
tag = TinyTag.get("dataset\fma_medium\102\102185.mp3")

OSError: [Errno 22] Invalid argument: 'dataset\x0cma_mediumBB185.mp3'